In [ ]:
# ============================================================================
# Brain-to-Text 2025: Standard Training Pipeline (Simplified)
# SECTION 1: HEADER AND DOCUMENTATION
# ============================================================================
"""
Complete training pipeline following the competition's standard approach:
1. Load neural data from HDF5 files
2. Train GRU encoder (without day-specific layers for simplicity)
3. CTC loss for phoneme prediction
4. Language model integration (N-gram + OPT-6.7B)
5. Word Error Rate (WER) evaluation

Based on the official competition baseline but simplified for easier replication.
Designed to run in Google Colab with minimal setup.
"""

"\nComplete training pipeline following the competition's standard approach:\n1. Load neural data from HDF5 files\n2. Train GRU encoder (without day-specific layers for simplicity)\n3. CTC loss for phoneme prediction\n4. Language model integration (N-gram + OPT-6.7B)\n5. Word Error Rate (WER) evaluation\n\nBased on the official competition baseline but simplified for easier replication.\nDesigned to run in Google Colab with minimal setup.\n"

# SECTION 2: IMPORTS AND SETUP

In [ ]:
import os
import sys
import json
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

# For CTC decoding and evaluation
try:
    import ctcdecode
    HAS_CTC_DECODE = True
except ImportError:
    print("⚠️  ctcdecode not available. Install with: pip install https://github.com/parlance/ctcdecode/archive/master.zip")
    HAS_CTC_DECODE = False

# For language model integration
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    import gc
    HAS_TRANSFORMERS = True
except ImportError:
    print("⚠️  transformers not available. Install with: pip install transformers accelerate")
    HAS_TRANSFORMERS = False

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")

⚠️  ctcdecode not available. Install with: pip install https://github.com/parlance/ctcdecode/archive/master.zip
🔧 Using device: cuda


# SECTION 3: CONSTANTS AND CONFIGURATION

In [ ]:
# Phoneme mapping from competition
PHONEME_TO_ID = {
    'BLANK': 0,    # CTC blank symbol
    'AA': 1, 'AE': 2, 'AH': 3, 'AO': 4, 'AW': 5,
    'AY': 6, 'B': 7, 'CH': 8, 'D': 9, 'DH': 10,
    'EH': 11, 'ER': 12, 'EY': 13, 'F': 14, 'G': 15,
    'HH': 16, 'IH': 17, 'IY': 18, 'JH': 19, 'K': 20,
    'L': 21, 'M': 22, 'N': 23, 'NG': 24, 'OW': 25,
    'OY': 26, 'P': 27, 'R': 28, 'S': 29, 'SH': 30,
    'T': 31, 'TH': 32, 'UH': 33, 'UW': 34, 'V': 35,
    'W': 36, 'Y': 37, 'Z': 38, 'ZH': 39,
    'SIL': 40,    # Silence token
}

ID_TO_PHONEME = {v: k for k, v in PHONEME_TO_ID.items()}
N_PHONEMES = len(PHONEME_TO_ID)

# Model configuration (based on competition baseline)
class Config:
    # Data parameters
    neural_dim = 512          # Number of neural features
    sample_rate = 50          # 20ms bins = 50Hz

    # Model architecture
    hidden_size = 768         # GRU hidden units
    num_layers = 5            # Number of GRU layers
    dropout = 0.4             # RNN dropout
    input_dropout = 0.2       # Input dropout

    # Training parameters
    batch_size = 32          # Reduced from 64 for memory efficiency
    learning_rate = 1e-3
    weight_decay = 1e-3
    num_epochs = 50
    grad_clip = 10.0

    # CTC parameters
    blank_id = 0              # CTC blank token

    # Evaluation parameters
    beam_width = 100          # Beam search width

    def __repr__(self):
        return f"Config(neural_dim={self.neural_dim}, hidden_size={self.hidden_size}, num_layers={self.num_layers})"

config = Config()

# SECTION 4: DATA LOADING AND DATASET CLASSES

In [ ]:
class Brain2TextDataset(Dataset):
    """
    Dataset for Brain-to-Text neural data
    Simplified version without day-specific handling
    """

    def __init__(self, data_dir: str, split: str = 'train', config: Config = config):
        """
        Initialize dataset

        Args:
            data_dir: Directory containing HDF5 files
            split: 'train', 'val', or 'test'
            config: Model configuration
        """
        self.data_dir = Path(data_dir)
        self.split = split
        self.config = config

        # Load all data
        self.samples = self._load_data()
        print(f"📊 Loaded {len(self.samples)} samples for {split} split")

    def _load_data(self) -> List[Dict]:
        """Load neural data from HDF5 files"""
        samples = []

        # Find HDF5 files matching the split
        all_hdf5_files = list(self.data_dir.rglob("*.hdf5"))
        if not all_hdf5_files:
            raise ValueError(f"No HDF5 files found in {self.data_dir}")

        # Filter files based on split (e.g., data_train.hdf5, data_val.hdf5, data_test.hdf5)
        split_patterns = [f"data_{self.split}", f"_{self.split}.", f"{self.split}_data"]
        hdf5_files = []
        for f in all_hdf5_files:
            fname_lower = f.name.lower()
            if any(pattern in fname_lower for pattern in split_patterns):
                hdf5_files.append(f)

        # If no split-specific files found, fall back to all files (with warning)
        if not hdf5_files:
            print(f"⚠️  No files matching split '{self.split}' found. Loading all HDF5 files.")
            hdf5_files = all_hdf5_files

        print(f"📁 Found {len(hdf5_files)} HDF5 files for '{self.split}' split")

        transcription_count = 0
        transcription_sources = {'sentence_label': 0, 'transcription': 0, 'sentenceText': 0, 'none': 0}

        for file_path in hdf5_files:
            print(f"Loading {file_path.name} for {self.split}...")

            with h5py.File(file_path, 'r') as f:
                # Debug: Show structure of first trial
                first_trial_key = list(f.keys())[0] if f.keys() else None
                if first_trial_key:
                    first_trial = f[first_trial_key]
                    print(f"   📋 First trial keys: {list(first_trial.keys())}")
                    print(f"   📋 First trial attrs: {list(first_trial.attrs.keys())}")

                for trial_key in f.keys():
                    trial_group = f[trial_key]

                    # Extract data
                    neural_features = trial_group['input_features'][:]  # (time_steps, 512)
                    n_time_steps = trial_group.attrs['n_time_steps']

                    # Skip samples without labels for training
                    if 'seq_class_ids' not in trial_group and self.split != 'test':
                        continue

                    phoneme_ids = trial_group['seq_class_ids'][:] if 'seq_class_ids' in trial_group else None
                    seq_len = trial_group.attrs['seq_len'] if 'seq_len' in trial_group.attrs else 0

                    # Extract text labels if available - try multiple possible field names
                    transcription = None
                    sentence_label = None

                    # Method 1: Try 'transcription' dataset
                    if 'transcription' in trial_group:
                        trans_data = trial_group['transcription'][:]
                        try:
                            if isinstance(trans_data, bytes):
                                transcription = trans_data.decode('utf-8').rstrip('\x00')
                            elif isinstance(trans_data, np.ndarray):
                                if trans_data.dtype.kind in ['S', 'U', 'O']:
                                    transcription = str(trans_data.item()) if trans_data.ndim == 0 else ''.join([str(x) for x in trans_data])
                                else:
                                    transcription = ''.join([chr(c) for c in trans_data if c != 0])
                            else:
                                transcription = str(trans_data)
                            transcription_sources['transcription'] += 1
                        except Exception as e:
                            pass

                    # Method 2: Try 'sentenceText' dataset (alternative field name)
                    if transcription is None and 'sentenceText' in trial_group:
                        try:
                            trans_data = trial_group['sentenceText'][:]
                            if isinstance(trans_data, bytes):
                                transcription = trans_data.decode('utf-8').rstrip('\x00')
                            elif isinstance(trans_data, np.ndarray):
                                transcription = ''.join([chr(c) for c in trans_data if c != 0])
                            else:
                                transcription = str(trans_data)
                            transcription_sources['sentenceText'] += 1
                        except:
                            pass

                    # Method 3: Try 'sentence_label' in attrs (preferred for WER)
                    if 'sentence_label' in trial_group.attrs:
                        try:
                            label_data = trial_group.attrs['sentence_label']
                            if isinstance(label_data, bytes):
                                sentence_label = label_data.decode('utf-8').rstrip('\x00')
                            elif isinstance(label_data, str):
                                sentence_label = label_data
                            else:
                                sentence_label = str(label_data)
                            transcription_sources['sentence_label'] += 1
                        except:
                            pass

                    # Method 4: Try 'sentenceText' in attrs
                    if sentence_label is None and 'sentenceText' in trial_group.attrs:
                        try:
                            label_data = trial_group.attrs['sentenceText']
                            if isinstance(label_data, bytes):
                                sentence_label = label_data.decode('utf-8').rstrip('\x00')
                            else:
                                sentence_label = str(label_data)
                        except:
                            pass

                    # Use sentence_label if available, otherwise transcription
                    final_transcription = sentence_label or transcription

                    if final_transcription:
                        transcription_count += 1
                    else:
                        transcription_sources['none'] += 1

                    sample = {
                        'neural_features': torch.FloatTensor(neural_features),
                        'n_time_steps': n_time_steps,
                        'phoneme_ids': torch.LongTensor(phoneme_ids) if phoneme_ids is not None else None,
                        'seq_len': seq_len,
                        'transcription': final_transcription,
                        'session': trial_group.attrs.get('session', 'unknown'),
                        'trial_key': trial_key
                    }

                    samples.append(sample)

        # Print transcription statistics
        print(f"📝 Transcription stats: {transcription_count}/{len(samples)} samples have transcriptions")
        print(f"   Sources: {transcription_sources}")

        return samples

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int) -> Dict:
        return self.samples[idx]

def collate_fn(batch: List[Dict]) -> Dict:
    """
    Collate function for DataLoader
    Handles variable-length sequences with padding
    """
    # Separate data
    neural_features = [sample['neural_features'] for sample in batch]
    n_time_steps = [sample['n_time_steps'] for sample in batch]
    phoneme_ids = [sample['phoneme_ids'] for sample in batch if sample['phoneme_ids'] is not None]
    seq_lens = [sample['seq_len'] for sample in batch]
    transcriptions = [sample['transcription'] for sample in batch]

    # Pad neural features
    neural_features_padded = pad_sequence(neural_features, batch_first=True, padding_value=0.0)

    # Pad phoneme sequences if available
    phoneme_ids_padded = None
    if phoneme_ids:
        phoneme_ids_padded = pad_sequence(phoneme_ids, batch_first=True, padding_value=0)

    return {
        'neural_features': neural_features_padded,  # (batch, max_time, 512)
        'n_time_steps': torch.LongTensor(n_time_steps),
        'phoneme_ids': phoneme_ids_padded,  # (batch, max_seq_len)
        'seq_lens': torch.LongTensor(seq_lens),
        'transcriptions': transcriptions
    }

# SECTION 5: MODEL ARCHITECTURE

In [ ]:
class SimplifiedGRUDecoder(nn.Module):
    """
    Simplified GRU decoder without day-specific layers
    Based on the competition baseline but streamlined
    """

    def __init__(self, config: Config):
        super().__init__()
        self.config = config

        # Input processing
        self.input_dropout = nn.Dropout(config.input_dropout)

        # Main GRU layers
        self.gru = nn.GRU(
            input_size=config.neural_dim,
            hidden_size=config.hidden_size,
            num_layers=config.num_layers,
            dropout=config.dropout if config.num_layers > 1 else 0,
            batch_first=True,
            bidirectional=False
        )

        # Initialize GRU weights
        for name, param in self.gru.named_parameters():
            if 'weight_hh' in name:
                nn.init.orthogonal_(param)
            elif 'weight_ih' in name:
                nn.init.xavier_uniform_(param)

        # Output projection to phonemes
        self.output_projection = nn.Linear(config.hidden_size, N_PHONEMES)
        nn.init.xavier_uniform_(self.output_projection.weight)

        # Learnable initial hidden state
        self.h0 = nn.Parameter(torch.zeros(config.num_layers, 1, config.hidden_size))
        nn.init.xavier_uniform_(self.h0)

    def forward(self, neural_features: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        """
        Forward pass

        Args:
            neural_features: (batch, time, 512) neural data
            lengths: (batch,) actual sequence lengths

        Returns:
            logits: (batch, time, n_phonemes) phoneme logits
        """
        batch_size = neural_features.size(0)

        # Apply input dropout
        x = self.input_dropout(neural_features)

        # Initialize hidden state
        h0 = self.h0.expand(self.config.num_layers, batch_size, self.config.hidden_size).contiguous()

        # GRU forward pass
        gru_output, _ = self.gru(x, h0)  # (batch, time, hidden_size)

        # Project to phoneme space
        logits = self.output_projection(gru_output)  # (batch, time, n_phonemes)

        return logits

# SECTION 6: TRAINING FUNCTIONS

In [ ]:
class Brain2TextTrainer:
    """
    Trainer class for the Brain-to-Text model
    """

    def __init__(self, model: nn.Module, config: Config):
        self.model = model
        self.config = config
        self.device = device

        # Move model to device
        self.model.to(self.device)

        # Loss function (CTC Loss)
        self.ctc_loss = nn.CTCLoss(blank=config.blank_id, reduction='mean', zero_infinity=True)

        # Optimizer
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=config.learning_rate,
            weight_decay=config.weight_decay
        )

        # Learning rate scheduler
        self.scheduler = CosineAnnealingLR(self.optimizer, T_max=config.num_epochs)

        # Training history
        self.train_losses = []
        self.val_losses = []

    def train_epoch(self, train_loader: DataLoader) -> float:
        """Train for one epoch"""
        self.model.train()
        total_loss = 0.0
        num_batches = 0

        for batch_idx, batch in enumerate(train_loader):
            # Move to device
            neural_features = batch['neural_features'].to(self.device)
            phoneme_ids = batch['phoneme_ids'].to(self.device) if batch['phoneme_ids'] is not None else None
            lengths = batch['n_time_steps'].to(self.device)
            seq_lens = batch['seq_lens'].to(self.device)

            # Skip batch if no labels
            if phoneme_ids is None:
                continue

            # Forward pass
            logits = self.model(neural_features, lengths)  # (batch, time, n_phonemes)

            # Prepare for CTC loss
            # CTC expects: (time, batch, n_classes)
            log_probs = F.log_softmax(logits, dim=-1).transpose(0, 1)

            # CTC loss
            loss = self.ctc_loss(
                log_probs=log_probs,
                targets=phoneme_ids,
                input_lengths=lengths,
                target_lengths=seq_lens
            )

            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.grad_clip)

            self.optimizer.step()

            total_loss += loss.item()
            num_batches += 1

            # Print progress
            if batch_idx % 50 == 0:
                print(f"  Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

        return total_loss / max(num_batches, 1)

    def validate(self, val_loader: DataLoader) -> float:
        """Validate the model"""
        self.model.eval()
        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
            for batch in val_loader:
                # Move to device
                neural_features = batch['neural_features'].to(self.device)
                phoneme_ids = batch['phoneme_ids'].to(self.device) if batch['phoneme_ids'] is not None else None
                lengths = batch['n_time_steps'].to(self.device)
                seq_lens = batch['seq_lens'].to(self.device)

                # Skip batch if no labels
                if phoneme_ids is None:
                    continue

                # Forward pass
                logits = self.model(neural_features, lengths)

                # CTC loss
                log_probs = F.log_softmax(logits, dim=-1).transpose(0, 1)
                loss = self.ctc_loss(log_probs, phoneme_ids, lengths, seq_lens)

                total_loss += loss.item()
                num_batches += 1

        return total_loss / max(num_batches, 1)

    def train(self, train_loader: DataLoader, val_loader: DataLoader = None) -> Dict:
        """
        Full training loop

        Returns:
            Dictionary with training history
        """
        print(f"🚀 Starting training for {self.config.num_epochs} epochs")
        print(f"📊 Model: {self.model}")
        print(f"⚙️  Config: {self.config}")

        best_val_loss = float('inf')

        for epoch in range(self.config.num_epochs):
            print(f"\n📅 Epoch {epoch+1}/{self.config.num_epochs}")

            # Train
            train_loss = self.train_epoch(train_loader)
            self.train_losses.append(train_loss)

            # Validate
            val_loss = None
            if val_loader is not None:
                val_loss = self.validate(val_loader)
                self.val_losses.append(val_loss)

                # Save best model
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    torch.save(self.model.state_dict(), 'best_model.pt')
                    print(f"💾 Saved best model (val_loss: {val_loss:.4f})")

            # Update learning rate
            self.scheduler.step()

            # Print epoch summary
            lr = self.optimizer.param_groups[0]['lr']
            val_loss_str = f"{val_loss:.4f}" if val_loss is not None else "N/A"
            print(f"📈 Train Loss: {train_loss:.4f}, Val Loss: {val_loss_str}, LR: {lr:.6f}")

        return {
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'best_val_loss': best_val_loss
        }

# SECTION 7: INFERENCE AND EVALUATION

In [ ]:
class Brain2TextInference:
    """
    Inference and evaluation for Brain-to-Text model
    """

    def __init__(self, model: nn.Module, config: Config):
        self.model = model
        self.config = config
        self.device = device

        # Move model to device and set to eval mode
        self.model.to(self.device)
        self.model.eval()

        # Initialize CTC decoder if available
        self.ctc_decoder = None
        if HAS_CTC_DECODE:
            labels = [ID_TO_PHONEME[i] for i in range(N_PHONEMES)]
            self.ctc_decoder = ctcdecode.CTCBeamDecoder(
                labels=labels,
                blank_id=config.blank_id,
                beam_width=config.beam_width
            )

    def decode_batch(self, neural_features: torch.Tensor, lengths: torch.Tensor) -> List[List[str]]:
        """
        Decode a batch of neural features to phoneme sequences

        Args:
            neural_features: (batch, time, 512) neural data
            lengths: (batch,) actual sequence lengths

        Returns:
            List of phoneme sequences for each sample in batch
        """
        with torch.no_grad():
            # Get model predictions
            logits = self.model(neural_features, lengths)  # (batch, time, n_phonemes)
            log_probs = F.log_softmax(logits, dim=-1)

            # Decode using CTC
            if self.ctc_decoder is not None:
                # Use beam search decoder
                beam_results, beam_scores, timesteps, out_seq_len = self.ctc_decoder.decode(log_probs, lengths)

                # Extract best hypothesis for each sample
                phoneme_sequences = []
                for i in range(beam_results.size(0)):
                    # Get best beam (index 0)
                    seq_len = out_seq_len[i][0]
                    phoneme_ids = beam_results[i][0][:seq_len].tolist()
                    phonemes = [ID_TO_PHONEME[pid] for pid in phoneme_ids if pid < len(ID_TO_PHONEME)]
                    phoneme_sequences.append(phonemes)

                return phoneme_sequences
            else:
                # Simple greedy decoding
                predictions = torch.argmax(log_probs, dim=-1)  # (batch, time)

                phoneme_sequences = []
                for i, length in enumerate(lengths):
                    pred_seq = predictions[i, :length].tolist()

                    # Remove blanks and consecutive duplicates (CTC decoding)
                    phonemes = []
                    prev_pid = None
                    for pid in pred_seq:
                        if pid != self.config.blank_id and pid != prev_pid:
                            if pid < len(ID_TO_PHONEME):
                                phonemes.append(ID_TO_PHONEME[pid])
                        prev_pid = pid

                    phoneme_sequences.append(phonemes)

                return phoneme_sequences

    def evaluate_dataset(self, data_loader: DataLoader) -> Dict:
        """
        Evaluate model on a dataset

        Returns:
            Dictionary with evaluation metrics
        """
        print("🔍 Evaluating model...")

        all_predictions = []
        all_targets = []
        all_transcriptions = []

        for batch in data_loader:
            # Move to device
            neural_features = batch['neural_features'].to(self.device)
            lengths = batch['n_time_steps'].to(self.device)
            transcriptions = batch['transcriptions']

            # Decode predictions
            predicted_phonemes = self.decode_batch(neural_features, lengths)

            # Store results
            all_predictions.extend(predicted_phonemes)
            all_transcriptions.extend(transcriptions)

            # Store targets if available
            if batch['phoneme_ids'] is not None:
                for i, seq_len in enumerate(batch['seq_lens']):
                    target_ids = batch['phoneme_ids'][i, :seq_len].tolist()
                    target_phonemes = [ID_TO_PHONEME[pid] for pid in target_ids if pid < len(ID_TO_PHONEME)]
                    all_targets.append(target_phonemes)

        # Calculate metrics
        results = {
            'num_samples': len(all_predictions),
            'predictions': all_predictions,
            'targets': all_targets if all_targets else None,
            'transcriptions': all_transcriptions
        }

        # Calculate phoneme accuracy if targets available
        if all_targets:
            phoneme_accuracy = self._calculate_phoneme_accuracy(all_predictions, all_targets)
            results['phoneme_accuracy'] = phoneme_accuracy
            print(f"📊 Phoneme Accuracy: {phoneme_accuracy:.3f}")

        return results

    def _calculate_phoneme_accuracy(self, predictions: List[List[str]], targets: List[List[str]]) -> float:
        """Calculate phoneme-level accuracy"""
        total_phonemes = 0
        correct_phonemes = 0

        for pred, target in zip(predictions, targets):
            # Align sequences (simple approach)
            min_len = min(len(pred), len(target))
            total_phonemes += max(len(pred), len(target))

            for i in range(min_len):
                if pred[i] == target[i]:
                    correct_phonemes += 1

        return correct_phonemes / max(total_phonemes, 1)

# SECTION 8: LANGUAGE MODEL INTEGRATION (FULL IMPLEMENTATION)

In [ ]:
def clear_gpu_memory():
    """Clear GPU memory for Colab optimization"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

class FullLanguageModel:
    """
    Full language model implementation using OPT-6.7B + comprehensive n-gram
    Optimized for Google Colab with memory management
    """

    def __init__(self, use_opt: bool = True, use_fallback: bool = True):
        self.use_opt = use_opt
        self.use_fallback = use_fallback
        self.device = device

        # Initialize comprehensive phoneme dictionary
        self.phoneme_dict = self._build_comprehensive_phoneme_dictionary()

        # Build n-gram patterns
        self.ngram_patterns = self._build_ngram_patterns()

        # Initialize OPT-6.7B
        self.opt_model = None
        self.opt_tokenizer = None
        self.fallback_model = None

        if HAS_TRANSFORMERS:
            self._initialize_language_models()
        else:
            print("⚠️  Transformers not available. Using basic phoneme mapping only.")

    def _initialize_language_models(self):
        """Initialize language models with memory optimization"""
        try:
            if self.use_opt:
                print("🤖 Loading OPT-6.7B model (this may take a few minutes)...")

                # Clear memory first
                clear_gpu_memory()

                # Load with memory optimization
                self.opt_tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
                self.opt_model = AutoModelForCausalLM.from_pretrained(
                    "facebook/opt-6.7b",
                    torch_dtype=torch.float16,  # Half precision to save VRAM
                    device_map="auto",  # Automatic device placement
                    low_cpu_mem_usage=True,  # Optimize CPU memory usage
                )

                # Ensure padding token
                if self.opt_tokenizer.pad_token is None:
                    self.opt_tokenizer.pad_token = self.opt_tokenizer.eos_token

                self.opt_model.eval()
                print("✅ OPT-6.7B loaded successfully")

        except Exception as e:
            print(f"⚠️  Failed to load OPT-6.7B: {e}")
            self.use_opt = False

        # Initialize fallback model if OPT fails or not requested
        if self.use_fallback and (not self.use_opt or self.opt_model is None):
            try:
                print("🤖 Loading fallback model (GPT-2 Medium)...")
                self.fallback_model = pipeline(
                    "text-generation",
                    model="gpt2-medium",
                    device=0 if torch.cuda.is_available() else -1,
                    torch_dtype=torch.float16
                )
                print("✅ Fallback model loaded successfully")
            except Exception as e:
                print(f"⚠️  Failed to load fallback model: {e}")
                self.fallback_model = None

    def _build_comprehensive_phoneme_dictionary(self) -> Dict:
        """Build comprehensive phoneme-to-word mapping"""
        return {
            # Articles and determiners
            ('DH', 'AH'): ['the'],
            ('AH'): ['a'],
            ('AE', 'N'): ['an'],

            # Common pronouns
            ('AY'): ['I'],
            ('Y', 'UW'): ['you'],
            ('HH', 'IY'): ['he'],
            ('SH', 'IY'): ['she'],
            ('IH', 'T'): ['it'],
            ('W', 'IY'): ['we'],
            ('DH', 'EY'): ['they'],

            # Common verbs
            ('IH', 'Z'): ['is'],
            ('AA', 'R'): ['are'],
            ('W', 'AA', 'Z'): ['was'],
            ('W', 'ER'): ['were'],
            ('HH', 'AE', 'V'): ['have'],
            ('HH', 'AE', 'Z'): ['has'],
            ('HH', 'AE', 'D'): ['had'],
            ('W', 'IH', 'L'): ['will'],
            ('K', 'AE', 'N'): ['can'],
            ('K', 'UH', 'D'): ['could'],
            ('SH', 'UH', 'D'): ['should'],
            ('W', 'UH', 'D'): ['would'],
            ('D', 'UW'): ['do'],
            ('D', 'AH', 'Z'): ['does'],
            ('D', 'IH', 'D'): ['did'],
            ('G', 'OW'): ['go'],
            ('K', 'AH', 'M'): ['come'],
            ('G', 'EH', 'T'): ['get'],
            ('M', 'EY', 'K'): ['make'],
            ('T', 'EY', 'K'): ['take'],
            ('S', 'IY'): ['see'],
            ('N', 'OW'): ['know'],
            ('TH', 'IH', 'NG', 'K'): ['think'],

            # Common nouns
            ('T', 'AY', 'M'): ['time'],
            ('P', 'ER', 'S', 'AH', 'N'): ['person'],
            ('Y', 'IH', 'R'): ['year'],
            ('W', 'EY'): ['way'],
            ('D', 'EY'): ['day'],
            ('M', 'AE', 'N'): ['man'],
            ('TH', 'IH', 'NG'): ['thing'],
            ('W', 'UH', 'M', 'AH', 'N'): ['woman'],
            ('L', 'AY', 'F'): ['life'],
            ('CH', 'AY', 'L', 'D'): ['child'],
            ('W', 'ER', 'L', 'D'): ['world'],
            ('S', 'K', 'UW', 'L'): ['school'],
            ('S', 'T', 'EY', 'T'): ['state'],
            ('F', 'AE', 'M', 'L', 'IY'): ['family'],
            ('S', 'T', 'UW', 'D', 'AH', 'N', 'T'): ['student'],
            ('G', 'R', 'UW', 'P'): ['group'],
            ('K', 'AH', 'N', 'T', 'R', 'IY'): ['country'],
            ('P', 'R', 'AA', 'B', 'L', 'AH', 'M'): ['problem'],
            ('HH', 'AE', 'N', 'D'): ['hand'],
            ('R', 'AY', 'T'): ['right'],
            ('S', 'IH', 'S', 'T', 'AH', 'M'): ['system'],

            # Prepositions and conjunctions
            ('IH', 'N'): ['in'],
            ('AA', 'N'): ['on'],
            ('AE', 'T'): ['at'],
            ('B', 'AY'): ['by'],
            ('F', 'AO', 'R'): ['for'],
            ('W', 'IH', 'TH'): ['with'],
            ('AH', 'V'): ['of'],
            ('T', 'UW'): ['to'],
            ('F', 'R', 'AH', 'M'): ['from'],
            ('AH', 'P'): ['up'],
            ('AW', 'T'): ['out'],
            ('AH', 'B', 'AW', 'T'): ['about'],
            ('IH', 'N', 'T', 'UW'): ['into'],
            ('OW', 'V', 'ER'): ['over'],
            ('AE', 'F', 'T', 'ER'): ['after'],
            ('AH', 'N', 'D'): ['and'],
            ('B', 'AH', 'T'): ['but'],
            ('AO', 'R'): ['or'],
            ('S', 'OW'): ['so'],
            ('IH', 'F'): ['if'],
            ('DH', 'AE', 'N'): ['than'],
            ('W', 'EH', 'N'): ['when'],
            ('W', 'EH', 'R'): ['where'],
            ('W', 'AY'): ['why'],
            ('HH', 'AW'): ['how'],

            # Common adjectives
            ('G', 'UH', 'D'): ['good'],
            ('N', 'UW'): ['new'],
            ('F', 'ER', 'S', 'T'): ['first'],
            ('L', 'AE', 'S', 'T'): ['last'],
            ('L', 'AO', 'NG'): ['long'],
            ('G', 'R', 'EY', 'T'): ['great'],
            ('L', 'IH', 'T', 'AH', 'L'): ['little'],
            ('OW', 'N'): ['own'],
            ('AH', 'DH', 'ER'): ['other'],
            ('OW', 'L', 'D'): ['old'],
            ('R', 'AY', 'T'): ['right'],
            ('B', 'IH', 'G'): ['big'],
            ('HH', 'AY'): ['high'],
            ('D', 'IH', 'F', 'R', 'AH', 'N', 'T'): ['different'],
            ('S', 'M', 'AO', 'L'): ['small'],
            ('L', 'AA', 'R', 'JH'): ['large'],
            ('N', 'EH', 'K', 'S', 'T'): ['next'],
            ('ER', 'L', 'IY'): ['early'],
            ('Y', 'AH', 'NG'): ['young'],
            ('IH', 'M', 'P', 'AO', 'R', 'T', 'AH', 'N', 'T'): ['important'],

            # Numbers
            ('W', 'AH', 'N'): ['one'],
            ('T', 'UW'): ['two'],
            ('TH', 'R', 'IY'): ['three'],
            ('F', 'AO', 'R'): ['four'],
            ('F', 'AY', 'V'): ['five'],
            ('S', 'IH', 'K', 'S'): ['six'],
            ('S', 'EH', 'V', 'AH', 'N'): ['seven'],
            ('EY', 'T'): ['eight'],
            ('N', 'AY', 'N'): ['nine'],
            ('T', 'EH', 'N'): ['ten'],

            # Medical/Clinical terms (relevant for neural data)
            ('P', 'EY', 'SH', 'AH', 'N', 'T'): ['patient'],
            ('D', 'AA', 'K', 'T', 'ER'): ['doctor'],
            ('N', 'ER', 'S'): ['nurse'],
            ('HH', 'AA', 'S', 'P', 'IH', 'T', 'AH', 'L'): ['hospital'],
            ('T', 'R', 'IY', 'T', 'M', 'AH', 'N', 'T'): ['treatment'],
            ('M', 'EH', 'D', 'AH', 'S', 'AH', 'N'): ['medicine'],
            ('S', 'ER', 'JH', 'ER', 'IY'): ['surgery'],
            ('TH', 'EH', 'R', 'AH', 'P', 'IY'): ['therapy'],
            ('D', 'AY', 'AH', 'G', 'N', 'OW', 'S', 'IH', 'S'): ['diagnosis'],
            ('S', 'IH', 'M', 'T', 'AH', 'M'): ['symptom'],
            ('K', 'AO', 'N', 'D', 'IH', 'SH', 'AH', 'N'): ['condition'],
            ('R', 'IH', 'K', 'AH', 'V', 'ER', 'IY'): ['recovery'],
            ('HH', 'EH', 'L', 'TH'): ['health'],
            ('K', 'EH', 'R'): ['care'],

            # Common phrases and contractions
            ('D', 'OW', 'N', 'T'): ["don't"],
            ('K', 'AE', 'N', 'T'): ["can't"],
            ('W', 'OW', 'N', 'T'): ["won't"],
            ('IH', 'T', 'S'): ["it's"],
            ('DH', 'AE', 'T', 'S'): ["that's"],
            ('W', 'AH', 'T', 'S'): ["what's"],
            ('HH', 'IY', 'Z'): ["he's"],
            ('SH', 'IY', 'Z'): ["she's"],
            ('DH', 'EH', 'R', 'Z'): ["there's"],
            ('HH', 'IH', 'R', 'Z'): ["here's"],
            ('L', 'EH', 'T', 'S'): ["let's"],
            ('AY', 'M'): ["I'm"],
            ('Y', 'UH', 'R'): ["you're"],
            ('W', 'IY', 'R'): ["we're"],
            ('DH', 'EY', 'R'): ["they're"],
            ('AY', 'V'): ["I've"],
            ('Y', 'UH', 'V'): ["you've"],
            ('W', 'IY', 'V'): ["we've"],
            ('DH', 'EY', 'V'): ["they've"],
            ('AY', 'L'): ["I'll"],
            ('Y', 'UH', 'L'): ["you'll"],
            ('HH', 'IY', 'L'): ["he'll"],
            ('SH', 'IY', 'L'): ["she'll"],
            ('W', 'IY', 'L'): ["we'll"],
            ('DH', 'EY', 'L'): ["they'll"],
        }

    def _build_ngram_patterns(self) -> Dict:
        """Build n-gram patterns for context-aware word prediction"""
        return {
            # Common 2-gram patterns
            ('the',): {
                'patient': 0.15, 'doctor': 0.08, 'hospital': 0.05, 'treatment': 0.04,
                'first': 0.06, 'last': 0.04, 'next': 0.03, 'same': 0.03,
                'right': 0.04, 'left': 0.03, 'best': 0.03, 'most': 0.05,
                'other': 0.04, 'new': 0.04, 'old': 0.03, 'young': 0.02,
                'time': 0.03, 'way': 0.03, 'world': 0.02, 'country': 0.02,
                'system': 0.02, 'problem': 0.03, 'question': 0.02, 'answer': 0.02
            },
            ('patient', 'was'): {
                'admitted': 0.25, 'discharged': 0.15, 'treated': 0.12, 'examined': 0.10,
                'diagnosed': 0.08, 'transferred': 0.06, 'stable': 0.05, 'critical': 0.04,
                'recovering': 0.08, 'improving': 0.07
            },
            ('the', 'patient'): {
                'was': 0.30, 'is': 0.25, 'has': 0.15, 'had': 0.10, 'will': 0.08,
                'needs': 0.05, 'requires': 0.04, 'underwent': 0.03
            },
            ('doctor', 'said'): {
                'the': 0.20, 'that': 0.30, 'he': 0.10, 'she': 0.10, 'it': 0.08,
                'we': 0.07, 'they': 0.05, 'this': 0.05, 'everything': 0.05
            },
            ('in', 'the'): {
                'hospital': 0.15, 'morning': 0.08, 'evening': 0.06, 'afternoon': 0.05,
                'room': 0.10, 'bed': 0.08, 'clinic': 0.06, 'office': 0.05,
                'emergency': 0.04, 'surgery': 0.05, 'recovery': 0.04, 'ward': 0.06,
                'future': 0.03, 'past': 0.02, 'present': 0.02, 'beginning': 0.03,
                'end': 0.03, 'middle': 0.02, 'center': 0.03, 'area': 0.03
            },
            ('will', 'be'): {
                'able': 0.15, 'ready': 0.10, 'available': 0.08, 'necessary': 0.06,
                'important': 0.05, 'difficult': 0.04, 'easy': 0.04, 'possible': 0.08,
                'discharged': 0.06, 'transferred': 0.04, 'treated': 0.05, 'examined': 0.04,
                'fine': 0.05, 'okay': 0.04, 'better': 0.06, 'worse': 0.03,
                'there': 0.05, 'here': 0.04, 'back': 0.04
            },
            ('I', 'am'): {
                'going': 0.15, 'not': 0.12, 'very': 0.08, 'so': 0.06, 'really': 0.05,
                'here': 0.06, 'there': 0.04, 'fine': 0.08, 'okay': 0.06, 'good': 0.07,
                'sorry': 0.05, 'happy': 0.04, 'sad': 0.03, 'tired': 0.04,
                'ready': 0.05, 'sure': 0.06
            },
            ('you', 'are'): {
                'going': 0.12, 'not': 0.10, 'very': 0.08, 'so': 0.06, 'really': 0.05,
                'right': 0.08, 'wrong': 0.04, 'correct': 0.05, 'welcome': 0.06,
                'fine': 0.08, 'okay': 0.06, 'good': 0.07, 'ready': 0.05,
                'sure': 0.06, 'here': 0.04, 'there': 0.04
            },
            ('this', 'is'): {
                'a': 0.20, 'the': 0.15, 'not': 0.10, 'very': 0.08, 'really': 0.06,
                'important': 0.08, 'good': 0.06, 'bad': 0.04, 'great': 0.05,
                'my': 0.05, 'our': 0.04, 'your': 0.04, 'his': 0.02, 'her': 0.03
            },
            ('it', 'is'): {
                'a': 0.15, 'the': 0.10, 'not': 0.12, 'very': 0.10, 'really': 0.08,
                'important': 0.08, 'good': 0.06, 'bad': 0.04, 'great': 0.05,
                'possible': 0.06, 'necessary': 0.05, 'difficult': 0.04, 'easy': 0.04,
                'time': 0.03
            }
        }

    def phonemes_to_text_candidates(self, phoneme_sequence: List[str], n_candidates: int = 5) -> List[str]:
        """
        Convert phoneme sequence to multiple text candidates using comprehensive dictionary
        """
        if not phoneme_sequence:
            return [""]

        candidates = set()

        # Method 1: Greedy longest-match decoding
        words = []
        i = 0
        while i < len(phoneme_sequence):
            matched = False
            # Try longest matches first (up to 8 phonemes)
            for length in range(min(8, len(phoneme_sequence) - i), 0, -1):
                phoneme_tuple = tuple(phoneme_sequence[i:i+length])
                if phoneme_tuple in self.phoneme_dict:
                    word_options = self.phoneme_dict[phoneme_tuple]
                    words.append(word_options[0])  # Take first option
                    i += length
                    matched = True
                    break

            if not matched:
                i += 1  # Skip unknown phoneme

        if words:
            base_text = ' '.join(words)
            candidates.add(base_text)

            # Generate variations by using alternative word choices
            for alt_idx in range(min(3, len(words))):
                alt_words = words.copy()
                # Try to find alternative for this position
                phoneme_start = 0
                for w_idx, word in enumerate(words):
                    if w_idx == alt_idx:
                        # Find phoneme sequence for this word
                        for phoneme_tuple, word_list in self.phoneme_dict.items():
                            if word in word_list and len(word_list) > 1:
                                # Use alternative word
                                alt_words[w_idx] = word_list[1 if word_list[0] == word else 0]
                                break
                        break

                alt_text = ' '.join(alt_words)
                if alt_text != base_text:
                    candidates.add(alt_text)

        # Method 2: Use n-gram patterns to generate contextual candidates
        if len(candidates) > 0:
            base_words = list(candidates)[0].split()

            # Try to extend or modify using n-gram patterns
            for i in range(len(base_words)):
                if i < len(base_words) - 1:
                    context = (base_words[i],)
                    if context in self.ngram_patterns:
                        for next_word, prob in list(self.ngram_patterns[context].items())[:3]:
                            new_words = base_words.copy()
                            if i + 1 < len(new_words):
                                new_words[i + 1] = next_word
                            else:
                                new_words.append(next_word)
                            candidates.add(' '.join(new_words))

        # Convert to list and ensure we have enough candidates
        candidate_list = list(candidates)

        # If we don't have enough candidates, create variations
        while len(candidate_list) < n_candidates and candidate_list:
            base = candidate_list[0]
            # Add simple variations
            variations = [
                base.replace(' the ', ' a '),
                base.replace(' is ', ' was '),
                base.replace(' are ', ' were '),
                base.replace(' will ', ' would '),
                base.replace(' can ', ' could ')
            ]

            for var in variations:
                if var != base and var not in candidate_list:
                    candidate_list.append(var)
                    if len(candidate_list) >= n_candidates:
                        break

        # Pad with empty strings if needed
        while len(candidate_list) < n_candidates:
            candidate_list.append(candidate_list[0] if candidate_list else "unknown")

        return candidate_list[:n_candidates]

    @torch.no_grad()
    def rescore_with_opt(self, text_candidates: List[str]) -> str:
        """
        Rescore text candidates using OPT-6.7B and return best one
        """
        if not text_candidates:
            return "unknown"

        # Filter out empty candidates
        valid_candidates = [c for c in text_candidates if c.strip()]
        if not valid_candidates:
            return "unknown"

        # If OPT is not available, use simple heuristics
        if not self.use_opt or self.opt_model is None:
            return self._simple_candidate_selection(valid_candidates)

        try:
            scores = []

            for text in valid_candidates:
                try:
                    # Tokenize with proper handling
                    inputs = self.opt_tokenizer(
                        text,
                        return_tensors="pt",
                        truncation=True,
                        max_length=256,  # Shorter for memory efficiency
                        padding=False
                    )

                    # Move to device
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}

                    # Get model predictions
                    with torch.cuda.amp.autocast():  # Use mixed precision
                        outputs = self.opt_model(**inputs)

                    # Calculate perplexity (lower is better, so we negate for scoring)
                    log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)

                    # Get probability of the actual tokens
                    input_ids = inputs['input_ids'][0]
                    total_log_prob = 0

                    for i in range(1, len(input_ids)):  # Skip first token
                        if i - 1 < log_probs.size(1):
                            token_log_prob = log_probs[0, i-1, input_ids[i]]
                            total_log_prob += token_log_prob.item()

                    # Average log probability (higher is better)
                    avg_log_prob = total_log_prob / max(len(input_ids) - 1, 1)

                    # Add length penalty (prefer reasonable length)
                    length_penalty = -abs(len(input_ids) - 10) * 0.01  # Prefer ~10 tokens

                    final_score = avg_log_prob + length_penalty
                    scores.append(final_score)

                except Exception as e:
                    print(f"Error scoring '{text[:50]}...': {e}")
                    scores.append(-float('inf'))

            # Return candidate with highest score
            if scores:
                best_idx = np.argmax(scores)
                return valid_candidates[best_idx]
            else:
                return valid_candidates[0]

        except Exception as e:
            print(f"Error in OPT rescoring: {e}")
            return self._simple_candidate_selection(valid_candidates)

    def _simple_candidate_selection(self, candidates: List[str]) -> str:
        """
        Simple candidate selection based on heuristics when OPT is not available
        """
        if not candidates:
            return "unknown"

        # Score candidates based on simple heuristics
        scores = []

        for candidate in candidates:
            score = 0
            words = candidate.lower().split()

            # Prefer reasonable length
            if 3 <= len(words) <= 15:
                score += 2
            elif 1 <= len(words) <= 20:
                score += 1

            # Prefer common words
            common_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}
            score += sum(1 for word in words if word in common_words)

            # Prefer medical/clinical terms if present
            medical_words = {'patient', 'doctor', 'hospital', 'treatment', 'medicine', 'therapy', 'diagnosis'}
            score += sum(2 for word in words if word in medical_words)

            # Penalize very short or very long candidates
            if len(words) < 2:
                score -= 1
            if len(words) > 20:
                score -= 2

            scores.append(score)

        # Return candidate with highest score
        best_idx = np.argmax(scores) if scores else 0
        return candidates[best_idx]

    def phonemes_to_text(self, phoneme_sequences: List[List[str]]) -> List[str]:
        """
        Convert phoneme sequences to text using full pipeline
        """
        import time
        from tqdm import tqdm

        texts = []
        start_time = time.time()

        print(f"🔄 Converting {len(phoneme_sequences)} phoneme sequences to text...")
        print("⚠️  This is VERY slow with OPT-6.7B (~3-5 seconds per sequence)")

        # Use tqdm for better progress tracking
        progress_bar = tqdm(phoneme_sequences, desc="Converting phonemes to text")

        for i, phonemes in enumerate(progress_bar):
            sequence_start = time.time()

            # Generate multiple candidates
            candidates = self.phonemes_to_text_candidates(phonemes, n_candidates=3)  # Reduced from 5

            # Rescore with language model
            best_text = self.rescore_with_opt(candidates)
            texts.append(best_text)

            sequence_time = time.time() - sequence_start

            # Update progress bar with timing info
            if i > 0:
                avg_time = (time.time() - start_time) / (i + 1)
                remaining = len(phoneme_sequences) - (i + 1)
                eta_minutes = (remaining * avg_time) / 60
                progress_bar.set_postfix({
                    'avg_time': f'{avg_time:.2f}s/seq',
                    'ETA': f'{eta_minutes:.1f}min'
                })

            # Early warning system
            if i == 4:  # After just 5 sequences
                elapsed = time.time() - start_time
                avg_time_per_seq = elapsed / 5
                if avg_time_per_seq > 3:  # If >3 seconds per sequence
                    total_eta_minutes = (len(phoneme_sequences) * avg_time_per_seq) / 60
                    print(f"\n⚠️  WARNING: Very slow processing detected after 5 samples!")
                    print(f"⚠️  Current speed: {avg_time_per_seq:.2f}s per sequence")
                    print(f"⚠️  Estimated total time: {total_eta_minutes:.0f} minutes ({total_eta_minutes/60:.1f} hours)")
                    print(f"💡 RECOMMENDATION: Stop and restart with skip_full_evaluation=True")
                    print(f"💡 Or use max_eval_samples=50 for faster results")

            # Periodic memory cleanup and progress updates
            if (i + 1) % 10 == 0:
                clear_gpu_memory()

        elapsed_total = time.time() - start_time
        print(f"✅ Phoneme-to-text conversion complete in {elapsed_total/60:.1f} minutes")
        print(f"📊 Average time per sequence: {elapsed_total/len(phoneme_sequences):.2f}s")

        return texts

# Simple fallback language model for basic functionality
class SimpleLanguageModel:
    """
    Simplified language model for phoneme-to-text conversion
    Fallback when full model is not available
    """

    def __init__(self):
        # Basic phoneme-to-word mapping
        self.phoneme_to_word = {
            ('DH', 'AH'): 'the',
            ('DH', 'AE', 'T'): 'that',
            ('DH', 'IH', 'S'): 'this',
            ('AH', 'N', 'D'): 'and',
            ('W', 'IH', 'TH'): 'with',
            ('F', 'AO', 'R'): 'for',
            ('Y', 'UW'): 'you',
            ('IH', 'T'): 'it',
            ('IH', 'N'): 'in',
            ('T', 'UW'): 'to',
            ('AH', 'V'): 'of',
            ('P', 'EY', 'SH', 'AH', 'N', 'T'): 'patient',
            ('D', 'AA', 'K', 'T', 'ER'): 'doctor',
            ('HH', 'AA', 'S', 'P', 'IH', 'T', 'AH', 'L'): 'hospital',
        }

    def phonemes_to_text(self, phoneme_sequences: List[List[str]]) -> List[str]:
        """Convert phoneme sequences to text using basic mapping"""
        texts = []

        for phonemes in phoneme_sequences:
            words = []
            i = 0

            while i < len(phonemes):
                matched = False
                for length in range(min(6, len(phonemes) - i), 0, -1):
                    phoneme_tuple = tuple(phonemes[i:i+length])
                    if phoneme_tuple in self.phoneme_to_word:
                        words.append(self.phoneme_to_word[phoneme_tuple])
                        i += length
                        matched = True
                        break

                if not matched:
                    i += 1

            texts.append(' '.join(words) if words else 'unknown')

        return texts

# SECTION 9: EVALUATION METRICS

In [ ]:
def calculate_wer(predictions: List[str], targets: List[str]) -> float:
    """
    Calculate Word Error Rate (WER)

    WER = (Substitutions + Insertions + Deletions) / Total_Words_in_Reference
    """
    if not targets or not predictions:
        return 1.0

    total_errors = 0
    total_words = 0

    for pred, target in zip(predictions, targets):
        if not target:
            continue

        pred_words = pred.lower().split()
        target_words = target.lower().split()

        total_words += len(target_words)

        # Simple edit distance calculation
        errors = abs(len(pred_words) - len(target_words))  # Insertions/deletions

        # Count substitutions
        min_len = min(len(pred_words), len(target_words))
        for i in range(min_len):
            if pred_words[i] != target_words[i]:
                errors += 1

        total_errors += errors

    return total_errors / max(total_words, 1)

# SECTION 10: MAIN TRAINING AND EVALUATION PIPELINE

In [ ]:
def main_training_pipeline(
    data_dir: str,
    output_dir: str = "/content/brain2text_output",
    max_epochs: int = 10,
    batch_size: int = None,
    learning_rate: float = 1e-3,
    use_simple_lm: bool = True,
    use_length_grouping: bool = True,
    bucket_boundaries: list = None,
    skip_full_evaluation: bool = False,
    max_eval_samples: int = 100
):
    """
    Complete training and evaluation pipeline with memory optimization

    Args:
        data_dir: Directory containing HDF5 files
        output_dir: Directory to save outputs
        max_epochs: Number of training epochs
        batch_size: Batch size (defaults to config value)
        learning_rate: Learning rate
        use_simple_lm: Whether to use simple language model
        use_length_grouping: Whether to use length-based grouping
        bucket_boundaries: Boundaries for length buckets
        skip_full_evaluation: Skip full language model evaluation (saves time)
        max_eval_samples: Maximum samples to evaluate with full LM
    """
    print("=" * 80)
    print("🧠 BRAIN-TO-TEXT 2025: MEMORY-OPTIMIZED TRAINING PIPELINE")
    print("=" * 80)

    # Update config with provided parameters
    if batch_size is not None:
        config.batch_size = batch_size
    config.learning_rate = learning_rate
    config.num_epochs = max_epochs

    if bucket_boundaries is None:
        bucket_boundaries = [500, 1000, 1500, 2000, 3000]

    print(f"🔧 Configuration:")
    print(f"   Batch size: {config.batch_size}")
    print(f"   Learning rate: {config.learning_rate}")
    print(f"   Max epochs: {max_epochs}")
    print(f"   Length grouping: {use_length_grouping}")
    print(f"   Bucket boundaries: {bucket_boundaries}")
    print(f"   Skip full evaluation: {skip_full_evaluation}")
    print(f"   Max eval samples: {max_eval_samples}")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load datasets using official train/val splits
    print("\n📁 Loading datasets...")
    train_dataset = Brain2TextDataset(data_dir, split='train', config=config)
    val_dataset = Brain2TextDataset(data_dir, split='val', config=config)

    print(f"📊 Train samples: {len(train_dataset)}")
    print(f"📊 Val samples: {len(val_dataset)}")

    if use_length_grouping:
        print("\n📏 Analyzing sequence lengths...")

        # Analyze sequence lengths for training data
        train_lengths = analyze_sequence_lengths(train_dataset)

        # Create length buckets for training data
        train_buckets = create_length_buckets(train_dataset, bucket_boundaries)

        # Create length buckets for validation data
        val_buckets = create_length_buckets(val_dataset, bucket_boundaries)

        # Create length-grouped data loaders
        print("\n🔄 Creating length-grouped data loaders...")
        train_loader = create_length_grouped_dataloader(
            train_dataset,
            config.batch_size,
            train_buckets,
            shuffle_buckets=True
        )

        val_loader = create_length_grouped_dataloader(
            val_dataset,
            config.batch_size,
            val_buckets,
            shuffle_buckets=False
        )

    else:
        # Use standard data loaders
        print("\n📦 Creating standard data loaders...")
        train_loader = DataLoader(
            train_dataset,
            batch_size=config.batch_size,
            shuffle=True,
            collate_fn=collate_fn,
            num_workers=0  # Avoid multiprocessing for memory efficiency
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=config.batch_size,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=0
        )

    # Initialize model
    print("\n🤖 Initializing model...")
    model = SimplifiedGRUDecoder(config)
    print(f"📈 Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Initialize trainer
    trainer = Brain2TextTrainer(model, config)

    # Train model
    print("\n🚀 Starting training...")
    training_history = trainer.train(train_loader, val_loader)

    # Save training history
    history_path = os.path.join(output_dir, 'training_history.json')
    with open(history_path, 'w') as f:
        json.dump(training_history, f, indent=2)

    # Plot training curves
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(training_history['train_losses'], label='Train Loss')
    if training_history['val_losses']:
        plt.plot(training_history['val_losses'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('CTC Loss')
    plt.title('Training Progress')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(training_history['train_losses'], label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('CTC Loss')
    plt.title('Training Loss (Log Scale)')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_curves.png'), dpi=150, bbox_inches='tight')
    plt.show()

    # Load best model for evaluation
    if os.path.exists('best_model.pt'):
        model.load_state_dict(torch.load('best_model.pt'))
        print("📥 Loaded best model for evaluation")

    # Initialize inference
    print("\n🔍 Starting evaluation...")
    inference = Brain2TextInference(model, config)

    # Evaluate on validation set
    val_results = inference.evaluate_dataset(val_loader)

    # Language model conversion (FULL IMPLEMENTATION)
    if skip_full_evaluation:
        print("\n⏭️  Skipping full language model evaluation (skip_full_evaluation=True)")
        print("🔄 Using simple language model for quick results...")
        language_model = SimpleLanguageModel()
        # Limit samples for faster evaluation
        eval_predictions = val_results['predictions'][:max_eval_samples]
        predicted_texts = language_model.phonemes_to_text(eval_predictions)
        print(f"✅ Evaluated {len(predicted_texts)} samples with simple language model")
    else:
        print("\n🤖 Initializing full language model...")
        try:
            language_model = FullLanguageModel(use_opt=True, use_fallback=True)
            # Limit samples to prevent excessive wait time
            eval_predictions = val_results['predictions'][:max_eval_samples]
            print(f"🔄 Evaluating {len(eval_predictions)} samples (limited from {len(val_results['predictions'])} total)")
            predicted_texts = language_model.phonemes_to_text(eval_predictions)
            print("✅ Using full language model with OPT-6.7B")
        except Exception as e:
            print(f"⚠️  Full language model failed: {e}")
            print("🔄 Falling back to simple language model...")
            language_model = SimpleLanguageModel()
            eval_predictions = val_results['predictions'][:max_eval_samples]
            predicted_texts = language_model.phonemes_to_text(eval_predictions)

    # Calculate WER if transcriptions available
    # Get transcriptions for the samples we actually evaluated (first max_eval_samples)
    eval_transcriptions = val_results['transcriptions'][:len(predicted_texts)]
    valid_transcriptions = [t for t in eval_transcriptions if t is not None and str(t).strip()]

    print(f"\n📊 Transcription analysis:")
    print(f"   Total predictions: {len(predicted_texts)}")
    print(f"   Transcriptions checked: {len(eval_transcriptions)}")
    print(f"   Valid transcriptions: {len(valid_transcriptions)}")
    if eval_transcriptions:
        sample_trans = eval_transcriptions[:3]
        print(f"   Sample transcriptions: {sample_trans}")

    wer = None  # Initialize WER

    if valid_transcriptions:
        # Filter to only include samples with valid transcriptions
        paired_data = [(pred, trans) for pred, trans in zip(predicted_texts, eval_transcriptions)
                       if trans is not None and str(trans).strip()]
        if paired_data:
            paired_predictions, paired_transcriptions = zip(*paired_data)
            wer = calculate_wer(list(paired_predictions), list(paired_transcriptions))
            print(f"📊 Word Error Rate (WER): {wer:.3f} ({wer*100:.1f}%)")
        else:
            print("⚠️  No valid prediction-transcription pairs found for WER calculation")
    else:
        print("⚠️  No valid transcriptions found in the data!")
        print("   This usually means transcription/sentence_label fields are missing from HDF5 files")

    # Save results
    results = {
        'wer': wer,
        'phoneme_accuracy': val_results.get('phoneme_accuracy', None),
        'num_samples': val_results['num_samples'],
        'config': config.__dict__
    }

    results_path = os.path.join(output_dir, 'evaluation_results.json')
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)

    # Save sample predictions
    sample_predictions = []
    for i in range(min(10, len(predicted_texts))):
        sample_predictions.append({
            'sample_id': i,
            'predicted_phonemes': val_results['predictions'][i],
            'predicted_text': predicted_texts[i],
            'target_text': eval_transcriptions[i] if i < len(eval_transcriptions) else None
        })

    samples_path = os.path.join(output_dir, 'sample_predictions.json')
    with open(samples_path, 'w') as f:
        json.dump(sample_predictions, f, indent=2)

    print(f"\n✅ Training complete! Results saved to: {output_dir}")
    print("=" * 80)

    # Return results including WER if calculated
    return_dict = {
        'model': model,
        'trainer': trainer,
        'inference': inference,
        'results': val_results
    }

    # Add WER if it was calculated
    if 'wer' in locals():
        return_dict['wer'] = wer
        print(f"📊 Final WER: {wer:.3f} ({wer*100:.1f}%)")

    return return_dict


# SECTION 11: ADVANCED PIPELINE FUNCTIONS

In [ ]:
def main_training_pipeline_full_lm(data_dir: str, output_dir: str = "brain2text_output_full"):
    """
    Training pipeline with guaranteed full language model integration
    Optimized for best performance with OPT-6.7B
    """
    print("=" * 80)
    print("🧠 BRAIN-TO-TEXT 2025: FULL LANGUAGE MODEL PIPELINE")
    print("=" * 80)

    # Check memory and provide recommendations
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"🔧 GPU Memory: {gpu_memory:.1f}GB")

        if gpu_memory < 12:
            print("⚠️  WARNING: GPU has <12GB memory. OPT-6.7B may not fit.")
            print("💡 RECOMMENDATION: Use Colab Pro or reduce to GPT-2 Medium")
        else:
            print("✅ GPU memory sufficient for OPT-6.7B")

    # Run standard training pipeline first
    results = main_training_pipeline(data_dir, output_dir)

    # Override with full language model for final evaluation
    print("\n" + "="*60)
    print("🚀 FINAL EVALUATION WITH FULL LANGUAGE MODEL")
    print("="*60)

    # Force full language model usage
    print("🤖 Loading full language model (OPT-6.7B)...")
    full_language_model = FullLanguageModel(use_opt=True, use_fallback=True)

    # Re-evaluate with full language model
    inference = results['inference']
    val_loader = None  # You'd need to pass this from main_training_pipeline

    # For now, use the existing results but with full LM
    predicted_texts_full = full_language_model.phonemes_to_text(results['results']['predictions'])

    # Calculate final WER
    # Get transcriptions matching the number of predictions
    all_transcriptions = results['results']['transcriptions'][:len(predicted_texts_full)]
    valid_transcriptions = [t for t in all_transcriptions if t is not None and str(t).strip()]

    wer_full = None
    if valid_transcriptions:
        # Filter to only include samples with valid transcriptions
        paired_data = [(pred, trans) for pred, trans in zip(predicted_texts_full, all_transcriptions)
                       if trans is not None and str(trans).strip()]
        if paired_data:
            paired_predictions, paired_transcriptions = zip(*paired_data)
            wer_full = calculate_wer(list(paired_predictions), list(paired_transcriptions))
            print(f"📊 Final Word Error Rate (WER) with OPT-6.7B: {wer_full:.3f} ({wer_full*100:.1f}%)")
        else:
            print("⚠️  No valid prediction-transcription pairs found for WER calculation")
    else:
        print("⚠️  No valid transcriptions found for WER calculation")

    # Save full results (always save, even if WER couldn't be calculated)
    full_results = {
        'wer_full_lm': wer_full,
        'predicted_texts_full': predicted_texts_full[:10],  # Save first 10 examples
        'model_used': 'OPT-6.7B' if full_language_model.opt_model else 'GPT-2-Medium',
        'config': config.__dict__
    }

    full_results_path = os.path.join(output_dir, 'full_lm_results.json')
    with open(full_results_path, 'w') as f:
        json.dump(full_results, f, indent=2)

    print(f"💾 Full LM results saved to: {full_results_path}")

    print("✅ Full language model evaluation complete!")
    return results

# SECTION 12: UTILITY FUNCTIONS

In [ ]:
def download_brain2text_data(data_dir: str = "/content/brain2text_data") -> str:
    """
    Download Brain-to-Text 2025 data directly from Dryad repository

    Args:
        data_dir: Directory to save downloaded data

    Returns:
        Path to downloaded data directory
    """
    import urllib.request
    import json
    import zipfile
    import tempfile
    from pathlib import Path

    print("🌐 Downloading Brain-to-Text 2025 data from Dryad...")
    print("⏳ This may take 10-30 minutes depending on your connection...")

    # Create data directory
    data_path = Path(data_dir)
    data_path.mkdir(parents=True, exist_ok=True)

    # Dryad repository information
    DRYAD_DOI = "10.5061/dryad.dncjsxm85"
    DRYAD_ROOT = "https://datadryad.org"
    urlified_doi = DRYAD_DOI.replace("/", "%2F")

    try:
        # Get file list from Dryad API
        print("📋 Fetching file list from Dryad API...")
        versions_url = f"{DRYAD_ROOT}/api/v2/datasets/doi:{urlified_doi}/versions"

        with urllib.request.urlopen(versions_url) as response:
            versions_info = json.loads(response.read().decode())

        files_url_path = versions_info["_embedded"]["stash:versions"][-1]["_links"]["stash:files"]["href"]
        files_url = f"{DRYAD_ROOT}{files_url_path}"

        with urllib.request.urlopen(files_url) as response:
            files_info = json.loads(response.read().decode())

        file_infos = files_info["_embedded"]["stash:files"]

        print(f"📁 Found {len(file_infos)} files to download")

        # Download each file
        downloaded_files = []
        for i, file_info in enumerate(file_infos):
            filename = file_info["path"]
            file_size = file_info.get("size", 0)

            # Skip README files
            if filename.lower() == "readme.md":
                continue

            print(f"📥 Downloading {filename} ({i+1}/{len(file_infos)}) - {file_size/1e6:.1f}MB...")

            download_path = file_info["_links"]["stash:download"]["href"]
            download_url = f"{DRYAD_ROOT}{download_path}"
            download_to_filepath = data_path / filename

            # Download with progress indication
            try:
                urllib.request.urlretrieve(download_url, download_to_filepath)
                downloaded_files.append(download_to_filepath)

                # Extract if it's a zip file
                if file_info["mimeType"] == "application/zip" or filename.endswith('.zip'):
                    print(f"📦 Extracting {filename}...")
                    with zipfile.ZipFile(download_to_filepath, "r") as zf:
                        zf.extractall(data_path)

                    # Remove zip file after extraction to save space
                    download_to_filepath.unlink()
                    print(f"🗑️  Removed {filename} after extraction")

            except Exception as e:
                print(f"⚠️  Failed to download {filename}: {e}")
                continue

        # Verify HDF5 files are present
        hdf5_files = list(data_path.rglob("*.hdf5"))
        h5_files = list(data_path.rglob("*.h5"))

        total_data_files = len(hdf5_files) + len(h5_files)

        if total_data_files == 0:
            print("⚠️  Warning: No HDF5 files found after download!")
            print("📁 Directory contents:")
            for item in data_path.iterdir():
                print(f"   {item.name}")
        else:
            print(f"✅ Download complete! Found {total_data_files} data files")
            print(f"📊 HDF5 files: {len(hdf5_files)}")
            print(f"📊 H5 files: {len(h5_files)}")

        # Print directory structure
        print(f"\n📁 Data saved to: {data_path}")
        print("📋 Directory structure:")
        for item in sorted(data_path.iterdir()):
            if item.is_dir():
                sub_files = list(item.iterdir())
                print(f"   📁 {item.name}/ ({len(sub_files)} files)")
            else:
                file_size = item.stat().st_size / 1e6
                print(f"   📄 {item.name} ({file_size:.1f}MB)")

        return str(data_path)

    except Exception as e:
        print(f"❌ Error downloading data: {e}")
        print("💡 Alternative: Upload your data manually to the data directory")
        print(f"💡 Expected location: {data_path}")
        return str(data_path)

def verify_data_structure(data_dir: str) -> bool:
    """
    Verify that the downloaded data has the expected structure

    Args:
        data_dir: Directory containing the data

    Returns:
        True if data structure is valid
    """
    data_path = Path(data_dir)

    print(f"🔍 Verifying data structure in {data_path}...")

    # Look for HDF5 files
    hdf5_files = list(data_path.rglob("*.hdf5"))
    h5_files = list(data_path.rglob("*.h5"))

    total_files = len(hdf5_files) + len(h5_files)

    if total_files == 0:
        print("❌ No HDF5 data files found!")
        return False

    print(f"✅ Found {total_files} data files")

    # Try to open one file and check structure
    test_file = hdf5_files[0] if hdf5_files else h5_files[0]

    try:
        with h5py.File(test_file, 'r') as f:
            keys = list(f.keys())

            if not keys:
                print(f"⚠️  File {test_file.name} is empty")
                return False

            # Check first trial structure
            first_trial = f[keys[0]]
            expected_keys = ['input_features', 'seq_class_ids', 'transcription']

            missing_keys = [key for key in expected_keys if key not in first_trial]

            if missing_keys:
                print(f"⚠️  Missing expected keys in {test_file.name}: {missing_keys}")
                print(f"📋 Available keys: {list(first_trial.keys())}")
            else:
                print("✅ Data structure looks correct")

            # Check data shapes
            neural_features = first_trial['input_features']
            print(f"📊 Neural features shape: {neural_features.shape}")
            print(f"📊 Expected: (time_steps, 512)")

            if neural_features.shape[1] != 512:
                print(f"⚠️  Unexpected feature dimension: {neural_features.shape[1]} (expected 512)")
                return False

            return True

    except Exception as e:
        print(f"❌ Error reading test file {test_file.name}: {e}")
        return False

# Utility function for memory management
def analyze_sequence_lengths(dataset):
    """
    Analyze sequence length distribution in the dataset

    Args:
        dataset: Brain2TextDataset instance or Subset

    Returns:
        List of sequence lengths
    """
    # Handle both Brain2TextDataset and Subset objects
    if hasattr(dataset, 'samples'):
        # Direct Brain2TextDataset
        samples = dataset.samples
    elif hasattr(dataset, 'dataset'):
        # Subset object from random_split
        samples = [dataset.dataset.samples[i] for i in dataset.indices]
    else:
        raise ValueError(f"Unsupported dataset type: {type(dataset)}")

    lengths = [sample['n_time_steps'] for sample in samples]

    print(f"📊 Sequence Length Analysis:")
    print(f"   Total samples: {len(lengths)}")
    print(f"   Min length: {min(lengths)}")
    print(f"   Max length: {max(lengths)}")
    print(f"   Mean length: {np.mean(lengths):.1f}")
    print(f"   Median length: {np.median(lengths):.1f}")
    print(f"   Std length: {np.std(lengths):.1f}")

    # Show percentiles
    percentiles = [25, 50, 75, 90, 95, 99]
    for p in percentiles:
        print(f"   {p}th percentile: {np.percentile(lengths, p):.0f}")

    return lengths

def create_length_buckets(dataset, bucket_boundaries=[500, 1000, 1500, 2000, 3000]):
    """
    Group samples into buckets by sequence length

    Args:
        dataset: Brain2TextDataset instance or Subset
        bucket_boundaries: List of length boundaries for bucketing

    Returns:
        Dictionary mapping bucket names to sample indices (relative to the dataset)
    """
    buckets = {}

    # Handle both Brain2TextDataset and Subset objects
    if hasattr(dataset, 'samples'):
        # Direct Brain2TextDataset
        samples = dataset.samples
        sample_indices = list(range(len(samples)))
    elif hasattr(dataset, 'dataset'):
        # Subset object from random_split
        samples = [dataset.dataset.samples[i] for i in dataset.indices]
        sample_indices = list(range(len(samples)))  # Use local indices for the subset
    else:
        raise ValueError(f"Unsupported dataset type: {type(dataset)}")

    # Create bucket names
    bucket_names = []
    for i in range(len(bucket_boundaries)):
        if i == 0:
            bucket_names.append(f"0-{bucket_boundaries[i]}")
        else:
            bucket_names.append(f"{bucket_boundaries[i-1]+1}-{bucket_boundaries[i]}")
    bucket_names.append(f"{bucket_boundaries[-1]+1}+")

    # Initialize buckets
    for name in bucket_names:
        buckets[name] = []

    # Sort samples into buckets
    for idx, sample in enumerate(samples):
        length = sample['n_time_steps']

        # Find appropriate bucket
        bucket_idx = len(bucket_boundaries)  # Default to largest bucket
        for i, boundary in enumerate(bucket_boundaries):
            if length <= boundary:
                bucket_idx = i
                break

        bucket_name = bucket_names[bucket_idx]
        buckets[bucket_name].append(idx)  # Use local index

    # Print bucket statistics (sorted by average length, largest first)
    print(f"📁 Length-based buckets (largest first):")
    non_empty_buckets = [(name, indices) for name, indices in buckets.items() if indices]

    # Sort by bucket name in reverse order (largest first)
    non_empty_buckets.sort(key=lambda x: x[0], reverse=True)

    for name, indices in non_empty_buckets:
        sample_lengths = [samples[i]['n_time_steps'] for i in indices]
        avg_length = np.mean(sample_lengths)
        max_length = max(sample_lengths)
        min_length = min(sample_lengths)
        print(f"   {name}: {len(indices)} samples (avg: {avg_length:.0f}, range: {min_length}-{max_length})")

    return buckets

class LengthGroupedSampler:
    """
    Sampler that groups by length and processes largest buckets first
    """
    def __init__(self, buckets, batch_size, shuffle_buckets=True, shuffle_within_bucket=True):
        self.batch_size = batch_size
        self.shuffle_buckets = shuffle_buckets
        self.shuffle_within_bucket = shuffle_within_bucket

        # Filter out empty buckets and sort by size (largest first)
        self.bucket_data = []
        for bucket_name, indices in buckets.items():
            if indices:  # Only non-empty buckets
                # Extract average length for sorting
                if '+' in bucket_name:
                    avg_length = float('inf')  # Largest bucket
                elif '-' in bucket_name:
                    # Get the upper bound of the range
                    avg_length = float(bucket_name.split('-')[1])
                else:
                    avg_length = 0
                self.bucket_data.append((avg_length, bucket_name, indices))

        # Sort by average length (largest first)
        self.bucket_data.sort(key=lambda x: x[0], reverse=True)

        print(f"🔄 Processing order (largest sequences first):")
        total_batches = 0
        for avg_length, bucket_name, indices in self.bucket_data:
            num_batches = len(indices) // batch_size
            total_batches += num_batches
            print(f"   {bucket_name}: {len(indices)} samples → {num_batches} full batches")

        print(f"📊 Total full batches: {total_batches}")
        self.total_batches = total_batches

    def __iter__(self):
        all_batches = []

        # Process each bucket (largest first)
        for avg_length, bucket_name, indices in self.bucket_data:
            bucket_indices = indices.copy()

            # Shuffle within bucket if requested
            if self.shuffle_within_bucket:
                import random
                random.shuffle(bucket_indices)

            # Create batches from this bucket
            for i in range(0, len(bucket_indices), self.batch_size):
                batch_indices = bucket_indices[i:i + self.batch_size]
                if len(batch_indices) == self.batch_size:  # Only full batches
                    all_batches.append((avg_length, batch_indices))

        # Optionally shuffle the order of batches while preserving size grouping
        if self.shuffle_buckets:
            import random
            random.shuffle(all_batches)

        # Yield batch indices
        for avg_length, batch_indices in all_batches:
            yield batch_indices

    def __len__(self):
        return self.total_batches

def create_length_grouped_dataloader(dataset, batch_size, buckets, shuffle_buckets=True):
    """
    Create DataLoader with length-based grouping

    Args:
        dataset: Brain2TextDataset instance or Subset
        batch_size: Batch size for training
        buckets: Dictionary from create_length_buckets
        shuffle_buckets: Whether to shuffle the order of buckets

    Returns:
        Custom DataLoader with length grouping
    """
    sampler = LengthGroupedSampler(buckets, batch_size, shuffle_buckets=shuffle_buckets)

    def length_grouped_collate_fn(batch_indices):
        """Collate function that works with batch indices"""
        # Handle both Brain2TextDataset and Subset objects
        if hasattr(dataset, 'samples'):
            # Direct Brain2TextDataset
            batch = [dataset.samples[i] for i in batch_indices]
        elif hasattr(dataset, 'dataset'):
            # Subset object from random_split
            batch = [dataset.dataset.samples[dataset.indices[i]] for i in batch_indices]
        else:
            raise ValueError(f"Unsupported dataset type: {type(dataset)}")

        return collate_fn(batch)

    class LengthGroupedDataLoader:
        """Custom DataLoader for length-grouped batches"""
        def __init__(self, dataset, sampler, collate_fn):
            self.dataset = dataset
            self.sampler = sampler
            self.collate_fn = collate_fn

        def __iter__(self):
            for batch_indices in self.sampler:
                yield self.collate_fn(batch_indices)

        def __len__(self):
            return len(self.sampler)

    return LengthGroupedDataLoader(dataset, sampler, length_grouped_collate_fn)

def optimize_for_colab():
    """
    Optimize settings for Google Colab environment
    """
    print("🔧 Optimizing for Google Colab...")

    # Clear any existing GPU memory
    clear_gpu_memory()

    # Set memory fraction if needed
    if torch.cuda.is_available():
        # Allow memory growth
        torch.cuda.empty_cache()

        # Print memory info
        allocated = torch.cuda.memory_allocated() / 1e9
        cached = torch.cuda.memory_reserved() / 1e9
        print(f"📊 GPU Memory - Allocated: {allocated:.2f}GB, Cached: {cached:.2f}GB")

    # Set environment variables for optimization
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

    print("✅ Colab optimization complete!")

# Quick test function
def quick_test_language_model():
    """
    Quick test of the language model functionality
    """
    print("🧪 Testing language model functionality...")

    # Test phoneme sequences
    test_phonemes = [
        ['DH', 'AH', 'P', 'EY', 'SH', 'AH', 'N', 'T'],  # "the patient"
        ['D', 'AA', 'K', 'T', 'ER', 'S', 'EH', 'D'],     # "doctor said"
        ['IH', 'N', 'DH', 'AH', 'HH', 'AA', 'S', 'P', 'IH', 'T', 'AH', 'L']  # "in the hospital"
    ]

    try:
        # Test full language model
        print("Testing FullLanguageModel...")
        full_lm = FullLanguageModel(use_opt=False, use_fallback=True)  # Skip OPT for quick test
        full_results = full_lm.phonemes_to_text(test_phonemes)

        print("Full LM Results:")
        for i, (phonemes, text) in enumerate(zip(test_phonemes, full_results)):
            print(f"  {i+1}. {phonemes} → '{text}'")

        # Test simple language model
        print("\nTesting SimpleLanguageModel...")
        simple_lm = SimpleLanguageModel()
        simple_results = simple_lm.phonemes_to_text(test_phonemes)

        print("Simple LM Results:")
        for i, (phonemes, text) in enumerate(zip(test_phonemes, simple_results)):
            print(f"  {i+1}. {phonemes} → '{text}'")

        print("✅ Language model test complete!")

    except Exception as e:
        print(f"❌ Language model test failed: {e}")
        import traceback
        traceback.print_exc()

# SECTION 13: USAGE EXAMPLES AND INSTRUCTIONS

In [ ]:
if __name__ == "__main__":
    print("""
    🧠 BRAIN-TO-TEXT 2025: STANDARD TRAINING PIPELINE

    This script implements the competition's standard approach with full data download capability:
    1. Automated data download from Dryad repository
    2. GRU encoder for neural feature processing
    3. CTC loss for phoneme prediction
    4. Full language model integration (OPT-6.7B + N-gram)
    5. Word Error Rate (WER) evaluation

    🎯 TWO-STEP EXECUTION PROCESS:

    ═══════════════════════════════════════════════════════════════════════════════
    📥 STEP 1: DATA DOWNLOAD AND VERIFICATION
    ═══════════════════════════════════════════════════════════════════════════════

    # Install dependencies first
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install transformers accelerate bitsandbytes optimum
    !pip install h5py numpy pandas scikit-learn tqdm matplotlib seaborn
    !pip install pyctcdecode

    # Download and verify data (10-30 minutes)
    data_dir = download_brain2text_data("/content/brain2text_data")
    data_valid = verify_data_structure(data_dir)

    if data_valid:
        print("✅ Data is ready for training!")
    else:
        print("❌ Data verification failed. Upload manually.")

    ═══════════════════════════════════════════════════════════════════════════════
    🏋️ STEP 2: TRAINING AND EVALUATION
    ═══════════════════════════════════════════════════════════════════════════════

    # 2A: Basic Training with Simple Language Model
    optimize_for_colab()
    quick_test_language_model()

    results_basic = main_training_pipeline(
        data_dir="/content/brain2text_data",
        output_dir="/content/brain2text_results",
        max_epochs=10,
        batch_size=32,
        use_simple_lm=True
    )

    # 2B: Advanced Training with Full Language Model (OPT-6.7B)
    # ⚠️  Requires >12GB GPU memory (Colab Pro recommended)
    results_full = main_training_pipeline_full_lm(
        data_dir="/content/brain2text_data",
        output_dir="/content/brain2text_results_full",
        max_epochs=10,
        batch_size=16  # Smaller batch for memory
    )

    🚀 QUICK START (Copy-Paste Ready):

    # Complete pipeline in one go
    data_dir = download_brain2text_data()
    if verify_data_structure(data_dir):
        optimize_for_colab()
        results = main_training_pipeline(
            data_dir=data_dir,
            max_epochs=5,
            batch_size=16,  # Reduced for memory efficiency
            use_length_grouping=True,  # Enable memory optimization
            skip_full_evaluation=False,  # Enable full evaluation to get WER
            max_eval_samples=100  # Limit evaluation samples for speed
        )
        print(f"Training complete! WER: {results.get('wer', 'N/A')}")

        # WER should now be calculated properly using official train/val splits

    EXPECTED PERFORMANCE:
    - Baseline WER: ~6.7% (competition baseline)
    - With Full LM WER: ~5.5% (estimated with OPT-6.7B)
    - Target WER: <5% (to beat 2024 winner)
    - Data download: ~10-30 minutes
    - Training time: ~2-4 hours on Colab GPU
    - LM Loading time: ~5-10 minutes (OPT-6.7B download)

    KEY CHANGES FOR WER CALCULATION:
    - Now uses official train/val splits (not random split)
    - Properly extracts sentence_label and transcription from HDF5 files
    - WER calculated on validation set with ground truth labels
    - Test set used only for final submission (no ground truth available)

    MEMORY REQUIREMENTS:
    - Training: ~8GB VRAM (GRU model)
    - OPT-6.7B: ~12.4GB VRAM (use Colab Pro for best results)
    - Total RAM: ~16GB (Colab Pro recommended)

    TROUBLESHOOTING:
    - Data download fails → Use manual upload alternative
    - Out of memory → Reduce batch_size, use gradient accumulation
    - CUDA errors → Restart runtime, check GPU availability

    SECTION REFERENCE:
    - Section 1-3: Setup and configuration
    - Section 4-5: Data loading and model architecture
    - Section 6-7: Training and inference
    - Section 8-9: Language model and evaluation
    - Section 10-11: Pipeline functions
    - Section 12: Utility functions (Data download, Colab optimization)
    - Section 13: This usage guide
    """)

    # Example usage (uncomment to run)
    # Step 1: Download data
    data_dir = download_brain2text_data("/content/brain2text_data")
    data_valid = verify_data_structure(data_dir)





    🧠 BRAIN-TO-TEXT 2025: STANDARD TRAINING PIPELINE

    This script implements the competition's standard approach with full data download capability:
    1. Automated data download from Dryad repository
    2. GRU encoder for neural feature processing
    3. CTC loss for phoneme prediction
    4. Full language model integration (OPT-6.7B + N-gram)
    5. Word Error Rate (WER) evaluation

    🎯 TWO-STEP EXECUTION PROCESS:

    ═══════════════════════════════════════════════════════════════════════════════
    📥 STEP 1: DATA DOWNLOAD AND VERIFICATION
    ═══════════════════════════════════════════════════════════════════════════════

    # Install dependencies first
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install transformers accelerate bitsandbytes optimum
    !pip install h5py numpy pandas scikit-learn tqdm matplotlib seaborn
    !pip install pyctcdecode

    # Download and verify data (10-30 minutes)
    data_dir = 

In [ ]:
data_dir = "/content/brain2text_data"

# Run training
optimize_for_colab()
results = main_training_pipeline(
    data_dir=data_dir,
    max_epochs=50,  # ⚠️ Use 50+ epochs! (5 epochs = 100% WER)
    batch_size=16,  # Reduced batch size for memory efficiency
    use_length_grouping=True,  # Enable length grouping
    skip_full_evaluation=False,  # Enable WER calculation
    max_eval_samples=100  # Reasonable number for evaluation
)
print(f"Training complete! WER: {results.get('wer', 'N/A')}")

# Expected results:
# - 50 epochs: ~20-40% WER
# - 100 epochs: ~10-15% WER
# - 150+ epochs: ~6-8% WER (competition baseline)


🔧 Optimizing for Google Colab...
📊 GPU Memory - Allocated: 0.00GB, Cached: 0.00GB
✅ Colab optimization complete!
🧠 BRAIN-TO-TEXT 2025: MEMORY-OPTIMIZED TRAINING PIPELINE
🔧 Configuration:
   Batch size: 16
   Learning rate: 0.001
   Max epochs: 50
   Length grouping: True
   Bucket boundaries: [500, 1000, 1500, 2000, 3000]
   Skip full evaluation: False
   Max eval samples: 100

📁 Loading datasets...
📁 Found 45 HDF5 files for 'train' split
Loading data_train.hdf5 for train...
   📋 First trial keys: ['input_features', 'seq_class_ids', 'transcription']
   📋 First trial attrs: ['block_num', 'n_time_steps', 'sentence_label', 'seq_len', 'session', 'trial_num']
Loading data_train.hdf5 for train...
   📋 First trial keys: ['input_features', 'seq_class_ids', 'transcription']
   📋 First trial attrs: ['block_num', 'n_time_steps', 'sentence_label', 'seq_len', 'session', 'trial_num']
Loading data_train.hdf5 for train...
   📋 First trial keys: ['input_features', 'seq_class_ids', 'transcription']
   📋